In [11]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 이미지와 레이블 데이터를 저장할 리스트
images = []
labels = []

# 데이터 디렉토리 설정
cropped_image_dir = r"D:\data\korean 01 data"
json_dir = r"D:\data\korean skin data\open data\data\Training\Label data\digiter cam"

# 데이터 로드
for id_folder in os.listdir(cropped_image_dir):
    image_folder_path = os.path.join(cropped_image_dir, id_folder)
    json_file_path = os.path.join(json_dir, id_folder, f"{id_folder}_01_F_01.json")

    if not os.path.exists(json_file_path):
        continue

    with open(json_file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    for image_name in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))  # 예시로 128x128로 크기 조정
        images.append(image)

        # 레이블 추출 (예: 이마의 색소침착)
        label = json_data['annotations']['forehead_pigmentation']
        labels.append(label)

# 이미지와 레이블을 NumPy 배열로 변환
images = np.array(images, dtype='float32') / 255.0  # 정규화
labels = np.array(labels)

# 레이블 원-핫 인코딩
labels = to_categorical(labels, num_classes=6)  

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 모델 설계
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(6, activation='softmax')
])

# 모델 컴파일

model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {test_acc}")


Epoch 1/10
22/22 [==============================] - 6s 243ms/step - loss: 1.3385 - accuracy: 0.4665 - val_loss: 1.2689 - val_accuracy: 0.5174
Epoch 2/10
22/22 [==============================] - 5s 231ms/step - loss: 1.2284 - accuracy: 0.5160 - val_loss: 1.2109 - val_accuracy: 0.5174
Epoch 3/10
22/22 [==============================] - 5s 236ms/step - loss: 1.2035 - accuracy: 0.5204 - val_loss: 1.1659 - val_accuracy: 0.5174
Epoch 4/10
22/22 [==============================] - 5s 232ms/step - loss: 1.2135 - accuracy: 0.4752 - val_loss: 1.1360 - val_accuracy: 0.5349
Epoch 5/10
22/22 [==============================] - 5s 233ms/step - loss: 1.1466 - accuracy: 0.5146 - val_loss: 1.2945 - val_accuracy: 0.5174
Epoch 6/10
22/22 [==============================] - 5s 234ms/step - loss: 1.1790 - accuracy: 0.5364 - val_loss: 1.1852 - val_accuracy: 0.4128
Epoch 7/10
22/22 [==============================] - 5s 233ms/step - loss: 1.1169 - accuracy: 0.5292 - val_loss: 1.1265 - val_accuracy: 0.5349
Epoch 

In [7]:
# 레이블 값 확인
unique_labels = np.unique(labels)
print(f"레이블 값들: {unique_labels}")

# 최대 레이블 값에 따라 num_classes 설정
num_classes = len(unique_labels)  # 레이블 값 개수를 num_classes로 설정

# 레이블 원-핫 인코딩
labels = to_categorical(labels, num_classes=num_classes)


레이블 값들: [0 1 2 3 4 5]
